In [ ]:
# Defining Custom Variables

number_of_training_steps = 1250
run_Name = "run2500-smaller"
model = "124M"
file_Name = "combined.txt"
file_path = "https://github.com/Varun-S-Nair/Trump-Tweet-Generator/raw/master/GPT-2Backend/realDonaldTrump.txt"

In [ ]:
%tensorflow_version 1.x
!pip3 install gpt-2-simple
import gpt_2_simple as gpt2
import tensorflow as tf
import os
import requests
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
  Created wheel for gpt-2-simple: filename=gpt_2_simple-0.7.1-cp36-none-any.whl size=23581 sha256=ea72bef9553f8c3f9abcb34e60f629bcaa18f9df6ee3fa3c552c7a0c92a95edc
  Stored in directory: /root/.cache/pip/wheels/0c/f8/23/b53ce437504597edff76bf9c3b8de08ad716f74f6c6baaa91a
Successfully built gpt-2-simple
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
gpt2.mount_gdrive()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!nvidia-smi

Tue Jun 16 15:48:48 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
model_name = model
if not os.path.isdir(os.path.join("models", model_name)):
	print(f"Downloading {model_name} model...")
	gpt2.download_gpt2(model_name=model_name)   # model is saved into current directory under /models/124M/

Fetching checkpoint: 1.05Mit [00:00, 357Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 72.5Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 326Mit/s]                                                    


Fetching model.ckpt.data-00000-of-00001: 498Mit [00:03, 128Mit/s]                                   
Fetching model.ckpt.index: 1.05Mit [00:00, 338Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 160Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 125Mit/s]                                                       


In [ ]:
file_name = tf.keras.utils.get_file(file_Name, file_path)

1114112/1108088 [==============================] - 0s 0us/step


## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              file_name,
              model_name=model_name,
              restore_from="fresh",
              run_name=run_Name,
              steps=number_of_training_steps)   # steps is max number of training steps
gpt2.copy_checkpoint_to_gdrive(run_name=run_Name, copy_folder=True)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:01<00:00,  1.88s/it]


dataset has 255621 tokens
Training...
[1 | 13.36] loss=3.72 avg=3.72
[2 | 17.88] loss=3.62 avg=3.67
[3 | 22.40] loss=3.50 avg=3.61
[4 | 26.90] loss=3.69 avg=3.63
[5 | 31.40] loss=3.61 avg=3.63
[6 | 35.91] loss=3.47 avg=3.60
[7 | 40.42] loss=3.55 avg=3.59
[8 | 44.92] loss=3.34 avg=3.56
[9 | 49.42] loss=3.38 avg=3.54
[10 | 53.91] loss=3.45 avg=3.53
[11 | 58.40] loss=3.41 avg=3.52
[12 | 62.89] loss=3.43 avg=3.51
[13 | 67.39] loss=3.39 avg=3.50
[14 | 71.87] loss=3.42 avg=3.50
[15 | 76.37] loss=3.31 avg=3.48
[16 | 80.85] loss=3.32 avg=3.47
[17 | 85.34] loss=3.26 avg=3.46
[18 | 89.83] loss=3.33 avg=3.45
[19 | 94.30] loss=3.29 avg=3.44
[20 | 98.77] loss=3.33 avg=3.43
[21 | 103.24] loss=3.21 avg=3.42
[22 | 107.72] loss=3.27 avg=3.42
[23 | 112.18] loss=3.06 avg=3.40
[24 | 116.65] loss=3.15 avg=3.39
[25 | 121.11] loss=3.08 avg=3.37
[26 | 125.57] loss=3.17 avg=3.36
[27 | 130.04] loss=3.05 avg=3.35
[28 | 134.52] loss=3.19 avg=3.34
[29 | 138.99] loss=3.18 avg=3.34
[30 | 143.45] loss=3.05 avg=3.33
[

FileExistsError: ignored

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [ ]:
gpt2.generate(sess, run_name=run_Name)

“The James Comey leaks are a total hoax, and the Economic Numbers” are higher than Jerry Nadler would have you believe. Comey was a fraud, as you may have heard. Consumer Confidence is at an all time high, unemployment is at a 17 year low and the Economy is BOOMING. Sorry, but I’m not going to prosecute anyone for the Collusion I’m witnessing is so bad for our Country that it’s amazing that I’m here to take it all in!” @foxandfriends
“I said the greatest influence on the election was the fact that Comey was able to lay the groundwork for the campaign of Donald Trump.” If you believe what I have to say, and I hope you do, and you’re right President Trump also created the possibility for Comey and his circle of people. All the while they are purposely trying to obstruct. They are purposely having fun!
The Fake News Media is hurting our Country with its fraudulent and highly inaccurate coverage of Mexico and the upcoming visit of @VP Mike Pence. The stories, ideas and reporting on these i

In [ ]:
import random
tf.reset_default_graph()
sess = gpt2.start_tf_sess()
# gpt2.copy_checkpoint_from_gdrive(run_name=run_Name, copy_folder=True)
gpt2.load_gpt2(sess, run_name=run_Name)
single_text = gpt2.generate(sess, run_name=run_Name, return_as_list=True)[0]
one_line = single_text.split("\n")[random.randint(0,len(arr)-1)]
print(single_text)

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
# gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

# gpt2.generate_to_file(sess,
#                       destination_path=gen_file,
#                       length=500,
#                       temperature=0.7,
#                       nsamples=100,
#                       batch_size=20
#                       )

In [ ]:
# # may have to run twice to get file to download
# files.download(gen_file)